<a href="https://colab.research.google.com/github/ondennis03/AAI2026/blob/main/Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 2 Code Generation with ReACT Prompting

ReACT-style loop (Plan → Code → Run → Observe)

Tools used: Google Colab + OpenAI API + ChatGPT

In [24]:
!pip -q install openai

In [25]:
import os
import re
from dataclasses import dataclass
from openai import OpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY") or ""
print("Key loaded:", bool(os.environ.get("OPENAI_API_KEY")))

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

MODEL_NAME = "gpt-4.1-mini"
TEMPERATURE = 0.2

@dataclass
class ReactResult:
    plan: str
    code: str
    run_line: str

def call_llm(prompt: str) -> str:
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=TEMPERATURE,
    )
    return resp.choices[0].message.content.strip()

def parse_react_response(text: str) -> ReactResult:
    plan_match = re.search(r"PLAN:\s*(.*?)\n\s*CODE:", text, flags=re.S | re.I)
    code_match = re.search(r"CODE:\s*```python\s*(.*?)```", text, flags=re.S | re.I)
    run_match  = re.search(r"RUN:\s*(.*)", text, flags=re.I)

    if not (plan_match and code_match and run_match):
        raise ValueError("Model output didn't match required PLAN/CODE/RUN format.")

    return ReactResult(
        plan=plan_match.group(1).strip(),
        code=code_match.group(1).strip(),
        run_line=run_match.group(1).strip(),
    )

TASK_DESCRIPTION = (
    "Write Python code that:\n"
    "1) Generates a small synthetic dataset of monthly sales for 3 product categories across 12 months.\n"
    "2) Calculates total sales, average monthly sales, and month-over-month growth (%).\n"
    "3) Prints a clean summary table.\n"
    "4) Creates ONE matplotlib line chart (no seaborn).\n"
    "5) Includes a main() and runs when executed.\n\n"
    "Constraints:\n"
    "- Use only: pandas, numpy, matplotlib\n"
    "- No external files or downloads\n"
    "- Keep the code readable and commented\n"
)

REACT_PROMPT = (
    "You are an assistant that writes Python code using a ReACT-style structure.\n\n"
    "You must follow this output format EXACTLY:\n\n"
    "PLAN:\n"
    "- <2 to 5 bullet points, brief and practical; do NOT write hidden reasoning>\n\n"
    "CODE:\n"
    "```python\n"
    "<full code here>\n"
    "```\n\n"
    "RUN:\n"
    "<one command that would run it in Colab>\n\n"
    "Task:\n"
    f"{TASK_DESCRIPTION}"
)

# Step A: Reason + Action
raw = call_llm(REACT_PROMPT)
print("\n=== MODEL RESPONSE (raw) ===\n")
print(raw)

result = parse_react_response(raw)
print("\n=== PLAN (from model) ===\n")
print(result.plan)

# Step B: Act (execute)
with open("main.py", "w", encoding="utf-8") as f:
    f.write(result.code + "\n")

print("\n=== RUNNING GENERATED CODE ===\n")
# Run command (fallback to python main.py)
if result.run_line.strip() == "python main.py":
    !python main.py
else:
    print("Model RUN line wasn't exactly 'python main.py'. Running main.py anyway.")
    !python main.py

Key loaded: True


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}